In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor

In [3]:
types = {'id' : 'int64',
        'item_nbr' : 'int32',  
        'store_nbr' : 'int8',
        'unit_sales' : 'float32', 
        'onpromotion' : 'object'}        

In [4]:
df_test = pd.read_csv(f'data/test.csv', parse_dates = ['date'], dtype= types, infer_datetime_format = True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_test.onpromotion.map({'False' : False, 'True' : True})
df_test.onpromotion = df_test.onpromotion.astype(bool)

df_test.describe(include='all')

,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,2
top,NaN,2017-08-27 00:00:00,NaN,NaN,False
freq,NaN,210654,NaN,NaN,3171867
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


In [9]:
import feather
df_subset = feather.read_dataframe('tmp/raw_groceries_subset')

In [13]:
df_subset = df_subset.drop(['index'], axis=1)
df_subset.tail()

,date,store_nbr,item_nbr,unit_sales,onpromotion
8125665,2017-08-15,54,2089339,4.0,False
8125666,2017-08-15,54,2106464,1.0,True
8125667,2017-08-15,54,2110456,192.0,False
8125668,2017-08-15,54,2113914,198.0,True
8125669,2017-08-15,54,2116416,2.0,False


In [22]:
dt_range = ['2017-07-16' , '2017-07-31']
train = df_subset[~df_subset['date'].isin(dt_range)]
valid = df_subset[df_subset['date'].isin(dt_range)]

In [24]:
%%time
add_datepart(train, 'date')
add_datepart(valid, 'date')

C:\Users\ranata\Anaconda3\envs\my_env\fastai\structured.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
C:\Users\ranata\Anaconda3\envs\my_env\fastai\structured.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
C:\Users\ranata\Anaconda3\envs\my_env\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

Wall time: 7.6 s


In [ ]:
#%time df_subset.describe(include='all')

In [ ]:
#%time df_subset.to_feather('tmp/processed_groceries_subset')

In [25]:
train.unit_sales = np.log1p(np.clip(train.unit_sales, 0, None))
valid.unit_sales = np.log1p(np.clip(valid.unit_sales, 0, None))

C:\Users\ranata\Anaconda3\envs\my_env\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
# df_subset.unit_sales = np.log1p(np.clip(df_subset.unit_sales, 0, None))

In [ ]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [26]:
# n_valid = len(df_test) + 5
# n_trn = len(df_subset) - n_valid
# train, valid = split_vals(df_subset, n_trn)
train.shape, valid.shape

((7907353, 17), (218317, 17))

In [27]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
           m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [28]:
y = np.array(train.unit_sales, dtype=np.float32)
trn = train.drop(['unit_sales'], axis=1)
y_val = np.array(valid.unit_sales, dtype=np.float32)
val = np.array(valid.drop(['unit_sales'], axis=1), dtype=np.float32)

In [29]:
%time x = np.array(trn, dtype = np.float32)

Wall time: 6.32 s


In [30]:
m = RandomForestRegressor(n_estimators=10, min_samples_leaf=100, n_jobs=4)
%time m.fit(x, y)
%time print_score(m)

Wall time: 2min 6s
[0.6584733312254023, 0.6858130333319725, 0.4269671890909639, 0.417090349153601]
Wall time: 12.8 s


In [31]:
m = RandomForestRegressor(n_estimators=50, min_samples_leaf=3, n_jobs=-1)
%time m.fit(x, y)
%time print_score(m)

Wall time: 12min 6s
[0.3610874946376816, 0.5086951319631764, 0.8276830476441468, 0.6792955837921277]
Wall time: 3min 20s
